In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [ ]:
# Farben und Labels für Szenarien
farben = {"RCP26": "#2ca02c", "RCP45": "#ff9900", "RCP85": "#e41a1c"}
labels = {"RCP26": "Best-Case Szenario", "RCP45": "Reference Szenario", "RCP85": "Worst-Case Szenario"}

In [ ]:
# Ordner mit deinen JSON-Dateien
pfad_zum_ordner = '../data/see_data_json'
# Liste aller Dateinamen (z. B. ["Maggiore.json", "Zugersee.json", ...])
dateien = [f for f in os.listdir(pfad_zum_ordner) if f.endswith(".json")]

# Funktion wie vorher
def extract_scenario_with_depth(data, depth, scenario, lake_name):
    d = data['yearly'][depth][scenario]
    df = pd.DataFrame({
        "year": d["x"],
        "temperature_avg": d["y_ave"],
        "temperature_min": d["y_min"],
        "temperature_max": d["y_max"],
        "scenario": scenario,
        "depth": depth,
        "lake": lake_name
    })
    return df

# Alle Seen & Szenarien kombinieren
alle_dfs = []
for datei in dateien:
    lake_name = datei.replace(".json", "")
    with open(os.path.join(pfad_zum_ordner, datei), "r") as f:
        data = json.load(f)
        for depth in ["surface", "bottom"]:
            for scenario in ["RCP26", "RCP45", "RCP85"]:
                df = extract_scenario_with_depth(data, depth, scenario, lake_name)
                alle_dfs.append(df)

df_alle_seen = pd.concat(alle_dfs, ignore_index=True)

df_alle_seen.sample(n=10)

In [ ]:
df_alle_seen["lake"].unique()

In [ ]:
#Fischdaten einlesen
df_fisch = pd.read_csv("../data/Fischdaten.csv")
df_fisch.sample(n=5)
display(df_fisch)

In [ ]:
#Spaltennamen Fischdaten ändern
df_fisch = df_fisch.rename(columns={
    'Name deutsch/lokal': 'Fisch', "L. Maggiore" : "Maggiore", "Lower L. Lugano" : "Lower-Lugano", "Upper L. Lugano" : "Upper-Lugano", 
    'L. Geneva' : "Geneva", 'Lower L. Constance': "Lower-Constance",
       'Upper L. Constance' : "Upper-Constance", 'Upper L. Zürich' : "Upper-Zurich", 'Lower L. Zürich':"Lower-Zurich", 'Walensee' : "Walen",
       'Rotsee' : "Rot", 'L. Biel' : "Biel", 'L. Murten': "Murten", 'L. Neuchâtel': "Neuchatel", 'L. Alpnach': "Lucerne-Alpnacher" ,
       'L. Lucerne, Gersauer Becken' :'Lucerne-Gersauer', 'L. Lucerne, Kreuztrichter' : 'Lucerne-Kreuztrichter' ,
       'L. Lucerne, Urnersee' : 'Lucerne-Urnersee' , 'Greifensee' : "Greifen", 'Pfäffikersee': 'Pfaffikon', 'L. Brienz': 'Brienz' ,
       'Klöntalersee' : 'Klontaler', 'Lago di Poschiavo': 'Poschiavo', 'Lac de Joux' : 'Joux' , "Lac de l'Hongrin" : 'LacdelHongrin',
       'L. Davos' : 'LakeDavos', 'Oeschinensee': 'Oeschinensee', 'L. St. Moritz':'StMoritz', 'L. Silvaplana':'Silvaplana', 'L. Sils' : "Sils"
})
df_fisch.head()

# Spalten anzeigen
#print(df_fisch.columns)

# Eindeutige Seenamen
#print(df_alle_seen['lake'].unique())

In [ ]:
# Schritt 1: Fischdaten vorbereiten (long-format)
id_vars = ['Fisch', 'Name wissenschaftlich', 'Einzugsgebiet', 'Gefährdungsstatus']
exclude = ['Kritische Temperatur °C', 'Minimale Temperatur °C', 'Kritischer O2-Grenzwert (mg/l)']
value_vars = [col for col in df_fisch.columns if col not in id_vars + exclude]

df_fisch_long = df_fisch.melt(
    id_vars=id_vars + exclude,
    value_vars=value_vars,
    var_name='lake',
    value_name='present'
)

# Nur Fische, die in einem See vorkommen
df_fisch_long = df_fisch_long[df_fisch_long['present'] == 'JA'].drop(columns='present')

df_fisch_long.head()

In [ ]:
# Schritt 2: Fischdaten mit den Seedaten kombinieren
# df_alle_seen sollte enthalten: lake, year, scenario, depth, temperature_max etc.
df_merged = pd.merge(df_fisch_long, df_alle_seen, on='lake', how='inner')

# Schritt 3: Kritische Temperatur als float
df_merged['Kritische Temperatur °C'] = pd.to_numeric(df_merged['Kritische Temperatur °C'], errors='coerce')

# Schritt 4: Überlebensstatus berechnen
df_merged['überlebt'] = df_merged['temperature_max'] < df_merged['Kritische Temperatur °C']

# Ergebnis anzeigen
df_merged.sample(n=6)

#df_merged.to_csv("df_merged.csv", index=False)

In [ ]:
temp_spalte = "temperature_avg"  # oder "temperature_max"

# 1. Fisch mit gültiger kritischer Temperatur & überlebt = False
fisch = df_merged[(df_merged['überlebt'] == False) & 
                  (df_merged['Kritische Temperatur °C'].notna())]['Fisch'].unique()[0]

# 1.2. See automatisch wählen (in dem dieser Fisch nicht überlebt)
see = df_merged[(df_merged['Fisch'] == fisch) &
                (df_merged['überlebt'] == False)]['lake'].unique()[0]

# 2. Filter: Nur surface-Tiefe, nur dieser Fisch & See
df_plot = df_merged[
    (df_merged['Fisch'] == fisch) &
    (df_merged['lake'] == see)
]

# 3. Gruppieren: Durchschnitt pro Jahr & Szenario (für saubere Linien)
df_plot_grouped = df_plot.groupby(["year", "scenario"]).agg({
    temp_spalte: "mean",
    "Kritische Temperatur °C": "first"
}).reset_index()

# 4. ⌀ Temperatur über alle Szenarien
df_temp_avg = df_plot_grouped.groupby("year")[temp_spalte].mean().reset_index()


# 6. Plot erstellen
plt.figure(figsize=(12,6))

# Hintergrundlinie: Durchschnitt über alle Szenarien
plt.plot(df_temp_avg["year"], df_temp_avg[temp_spalte],
         color="gray", linestyle=":", label="⌀ max. Temperatur")

# Hauptlinien: Szenarien + Beschriftung + Sterbejahr
for scenario, group in df_plot_grouped.groupby("scenario"):
    # sortieren nach Jahr
    group_sorted = group.sort_values("year")

    # Linie zeichnen
    plt.plot(group_sorted["year"], group_sorted[temp_spalte],
             label=labels.get(scenario, scenario),
             color=farben.get(scenario), linewidth=2)

    # Szenario am Linienende beschriften
    x_end = group_sorted["year"].max()
    y_end = group_sorted[group_sorted["year"] == x_end][temp_spalte].values[0]
    plt.text(x_end + 1, y_end, labels.get(scenario, scenario), color=farben.get(scenario))

    # Sterbejahr berechnen (erstes Jahr mit max. Temp > kritischer Temp.)
    krit_temp = group_sorted["Kritische Temperatur °C"].iloc[0]
    dead_years = group_sorted[group_sorted[temp_spalte] > krit_temp]

    if not dead_years.empty:
        year_dead = dead_years["year"].iloc[0]
        temp_dead = dead_years[temp_spalte].iloc[0]

        # Punkt markieren
        plt.scatter(year_dead, temp_dead, color=farben[scenario], s=70, zorder=5)
        # Text schreiben
        plt.text(year_dead + 1, temp_dead + 0.3, f"stirbt ab {year_dead}", color=farben[scenario])
# Kritische Temperatur als horizontale Linie
krit_temp = df_plot_grouped["Kritische Temperatur °C"].iloc[0]
plt.axhline(krit_temp, color="red", linestyle="--", label="Kritische Temp.")

# Beschriftung & Layout
plt.title(f"{fisch} in {see} – Temperaturentwicklung nach Szenario")
plt.xlabel("Jahr")
plt.ylabel("Wassertemperatur [°C]")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 1. Fisch mit gültiger kritischer Temperatur & überlebt = False
fisch = df_merged[(df_merged['überlebt'] == False) & 
                  (df_merged['Kritische Temperatur °C'].notna())]['Fisch'].unique()[0]

# 2. See dazu ermitteln
see = df_merged[(df_merged['Fisch'] == fisch) & 
                (df_merged['überlebt'] == False)]['lake'].unique()[0]

# 3. Filter: Fisch + See, beide Tiefen
df_plot = df_merged[
    (df_merged['Fisch'] == fisch) &
    (df_merged['lake'] == see) &
    (df_merged["depth"] == "surface")
]

# 4. Gruppieren: nach Jahr, Szenario, Tiefe
df_plot_grouped = df_plot.groupby(["year", "scenario", "depth"]).agg({
    "temperature_max": "mean",
    "Kritische Temperatur °C": "first"
}).reset_index()


# 6. Plot vorbereiten
plt.figure(figsize=(12, 6))

# Linienplot für jede Tiefe + Szenario
for (scenario, depth), group in df_plot_grouped.groupby(["scenario", "depth"]):
    group_sorted = group.sort_values("year")
    linestyle = "-" if depth == "surface" else "--"
    label = f"{labels.get(scenario, scenario)} ({depth})"
    
    # Linie zeichnen
    plt.plot(group_sorted["year"], group_sorted["temperature_max"],
             label=label, color=farben.get(scenario), linestyle=linestyle, linewidth=2)
    
    # Sterbejahr markieren
    krit_temp = group_sorted["Kritische Temperatur °C"].iloc[0]
    dead = group_sorted[group_sorted["temperature_max"] > krit_temp]
    if not dead.empty:
        year_dead = dead["year"].iloc[0]
        temp_dead = dead["temperature_max"].iloc[0]
        plt.scatter(year_dead, temp_dead, color=farben[scenario], s=60)
        plt.text(year_dead + 1, temp_dead + 0.2, f"{depth}: ab {year_dead}", color=farben[scenario], fontsize=9)

# Horizontale Linie: Kritische Temperatur
plt.axhline(krit_temp, color="red", linestyle=":", label=f"Kritische Temp. ({krit_temp}°C)")

# Layout
plt.title(f"{fisch} in {see} – Temperaturentwicklung in surface & bottom")
plt.xlabel("Jahr")
plt.ylabel("Max. Wassertemperatur [°C]")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
temp_spalte = "temperature_avg"  # oder "temperature_max"

# 1. Zufälliger Fisch aus dem gesamten DataFrame
alle_fische = df_merged['Fisch'].dropna().unique()
fisch = random.choice(alle_fische)

# 2. Zufälliger See, in dem dieser Fisch vorkommt
seen_vom_fisch = df_merged[df_merged['Fisch'] == fisch]['lake'].dropna().unique()
see = random.choice(seen_vom_fisch)

# 3. Filter für surface-Tiefe
df_plot = df_merged[
    (df_merged["Fisch"] == fisch) &
    (df_merged["lake"] == see) &
    (df_merged["depth"] == "surface") &
    (df_merged["Kritische Temperatur °C"].notna())
]

# Wenn keine Daten da sind (z. B. keine Kritische Temperatur oder Surface fehlt), überspringen
if df_plot.empty:
    print(f"Keine Daten für {fisch} in {see} – nächster Versuch.")
else:
    # Gruppieren
    df_plot_grouped = df_plot.groupby(["year", "scenario"]).agg({
        temp_spalte: "mean",
        "Kritische Temperatur °C": "first"
    }).reset_index()

    df_temp_avg = df_plot_grouped.groupby("year")[temp_spalte].mean().reset_index()


    krit_temp = df_plot_grouped["Kritische Temperatur °C"].iloc[0]

    # Plot
    plt.figure(figsize=(12,6))
    plt.plot(df_temp_avg["year"], df_temp_avg[temp_spalte],
             color="gray", linestyle=":", label="⌀ max. Temperatur")

    for scenario, group in df_plot_grouped.groupby("scenario"):
        group_sorted = group.sort_values("year")
        plt.plot(group_sorted["year"], group_sorted[temp_spalte],
                 label=labels.get(scenario, scenario),
                 color=farben.get(scenario), linewidth=2)

        # Szenarienname ans Ende
        x_end = group_sorted["year"].max()
        y_end = group_sorted[group_sorted["year"] == x_end][temp_spalte].values[0]
        plt.text(x_end + 1, y_end, labels.get(scenario, scenario), color=farben.get(scenario))

        # Aussterben markieren
        dead_years = group_sorted[group_sorted[temp_spalte] > krit_temp]
        if not dead_years.empty:
            year_dead = dead_years["year"].iloc[0]
            temp_dead = dead_years[temp_spalte].iloc[0]
            plt.scatter(year_dead, temp_dead, color=farben[scenario], s=70, zorder=5)
            plt.text(year_dead + 1, temp_dead + 0.3, f"stirbt ab {year_dead}", color=farben[scenario])

    plt.axhline(krit_temp, color="red", linestyle="--", label=f"Kritische Temp. ({krit_temp}°C)")
    plt.title(f"{fisch} in {see} – Temperaturentwicklung nach Szenario")
    plt.xlabel("Jahr")
    plt.ylabel("Max. Wassertemperatur [°C]")
    plt.legend()
    plt.xlim(right=df_plot_grouped["year"].max() + 10)
    plt.tight_layout()
    plt.show()

interessant: adriatische forelle in upper-lugano

Frage 1: wie gehe ich mit den missing temperatur werten um ?
Frage 2: temperatur max oder temperatur avg
Frage 3: depth or surface? : 
„In dieser Analyse gehen wir davon aus, dass alle Fischarten potenziell in beiden Tiefenzonen des Sees vorkommen – auch wenn einige biologisch bevorzugt an bestimmten Stellen leben. Ziel ist es, den Einfluss des Klimawandels in verschiedenen Seeschichten vergleichbar zu machen.“



In [ ]:
# Anzahl Fischarten, die aussterben je nach Temperaturtyp
def aussterbevergleich(df, temp_col):
    df_check = df.copy()
    df_check["überlebt"] = df_check[temp_col] < df_check["Kritische Temperatur °C"]
    aussterbe_fische = df_check[df_check["überlebt"] == False]["Fisch"].nunique()
    return aussterbe_fische

# Beide Temperaturarten vergleichen
anzahl_max = aussterbevergleich(df_merged, "temperature_max")
anzahl_avg = aussterbevergleich(df_merged, "temperature_avg")

# Ergebnis anzeigen
print(f"Fische sterben bei temperature_max: {anzahl_max}")
print(f"Fische sterben bei temperature_avg: {anzahl_avg}")

#macht gar keinen sinn!

In [ ]:
df_deaths = df_merged[df_merged["temperature_avg"] > df_merged["Kritische Temperatur °C"]]
df_deaths_per_year = df_deaths.groupby("year")["Fisch"].nunique()
df_deaths_per_year.plot(title="Anzahl betroffener Fischarten pro Jahr", ylabel="Fischarten")

In [ ]:
#Für jeden See: Welche Fischarten sind mindestens einmal vom Aussterben betroffen?

# 1. Alle Fisch-See-Kombis, bei denen überlebt == False
df_betroffen = df_merged[df_merged["überlebt"] == False][["lake", "Fisch"]].drop_duplicates()

# 2. Gruppieren nach See
df_pro_see = df_betroffen.groupby("lake")["Fisch"].apply(list).reset_index()

# 3. Spalte mit Anzahl der betroffenen Arten
df_pro_see["Anzahl betroffener Fische"] = df_pro_see["Fisch"].apply(len)

# 4. Sortieren (z. B. nach Anzahl, absteigend)
df_pro_see = df_pro_see.sort_values("Anzahl betroffener Fische", ascending=False)

# Anzeigen
df_pro_see

#die umkehrung muss auch untersucht werden

In [ ]:
# 1. Alle Fälle, wo ein Fisch nicht überlebt
df_betroffen = df_merged[df_merged["überlebt"] == False][["Fisch", "lake"]].drop_duplicates()

# 2. Gruppieren: Anzahl betroffener Seen pro Fisch
df_fisch_stats = df_betroffen.groupby("Fisch")["lake"].nunique().reset_index()
df_fisch_stats.columns = ["Fisch", "Anzahl betroffener Seen"]

# 3. Sortieren (meistbetroffen oben)
df_fisch_stats = df_fisch_stats.sort_values("Anzahl betroffener Seen", ascending=False)

# Anzeigen
df_fisch_stats

In [ ]:
#Wie viele verschiedene Fischarten sterben pro Szenario mindestens in einem See aus?

# 1. Alle Fälle, wo Fische nicht überleben
df_betroffen = df_merged[df_merged["überlebt"] == False]

# 2. Für jedes Szenario: Anzahl einzigartiger Fischarten
df_szenario_summary = df_betroffen.groupby("scenario")["Fisch"].nunique().reset_index()
df_szenario_summary.columns = ["Szenario", "Anzahl betroffener Fischarten"]

# 3. Sortieren nach Szenario
df_szenario_summary = df_szenario_summary.sort_values("Szenario")

# Anzeigen
display(df_szenario_summary)

In [ ]:
# 1. Alle Kombinationen, in denen der Fisch nicht überlebt
df_betroffen = df_merged[df_merged["überlebt"] == False]

# 2. Gruppieren: Für jedes Szenario & jeden See → Anzahl betroffener Fischarten
df_szenario_see = df_betroffen.groupby(["scenario", "lake"])["Fisch"].nunique().reset_index()
df_szenario_see.columns = ["Szenario", "See", "Anzahl betroffener Fischarten"]

# 3. Sortieren
df_szenario_see = df_szenario_see.sort_values(["Szenario", "Anzahl betroffener Fischarten"], ascending=[True, False])

# 4. Anzeigen
display(df_szenario_see)

In [ ]:
# 1. Nur surface-Zeilen behalten
df_surface = df_merged[df_merged["depth"] == "surface"]

# 2. Fische, die mindestens in einem See (unter surface-Bedingung) aussterben
# Nur Fische, die mind. 1x nicht überleben UND kritische Temperatur haben
fische_bedroht = df_surface[
    (df_surface["überlebt"] == False) &
    (df_surface["Kritische Temperatur °C"].notna())
]["Fisch"].unique()

# 3. Alle surface-Zeilen dieser Fische (auch wenn sie in manchen Seen überleben)
df_fische_surface = df_surface[df_surface["Fisch"].isin(fische_bedroht)]

# 4. Gesamtanzahl unterschiedlicher Seen pro Fisch
df_gesamt = df_fische_surface[["Fisch", "lake"]].drop_duplicates()
df_gesamt_stats = df_gesamt.groupby("Fisch")["lake"].nunique().reset_index()
df_gesamt_stats.columns = ["Fisch", "Anzahl Gesamt-Seen"]

# 5. Anzahl Seen, in denen Fisch min. einmal ausstirbt
df_bedroht = df_fische_surface[df_fische_surface["überlebt"] == False][["Fisch", "lake"]].drop_duplicates()
df_bedroht_stats = df_bedroht.groupby("Fisch")["lake"].nunique().reset_index()
df_bedroht_stats.columns = ["Fisch", "Anzahl bedrohter Seen"]

# 6. Kritische Temperatur pro Fisch (einmalig)
df_krit_temp = df_fische_surface.dropna(subset=["Kritische Temperatur °C"])[["Fisch", "Kritische Temperatur °C"]].drop_duplicates("Fisch")

# 7. Alles zusammenführen
df_final = pd.merge(df_gesamt_stats, df_bedroht_stats, on="Fisch", how="left")
df_final = pd.merge(df_final, df_krit_temp, on="Fisch", how="left")
df_final["Anzahl bedrohter Seen"] = df_final["Anzahl bedrohter Seen"].fillna(0).astype(int)

# Tabelle anzeigen
df_final

In [ ]:
df_alle_seen

In [ ]:
df_adriatische_forelle = df_merged[df_merged["Fisch"] == "Adriatische Forelle"].copy()

spalten_entfernen = [
    "Einzugsgebiet",
    "Minimale Temperatur °C",
    "temperature_min",
    "temperature_max",
    "Kritischer O2-Grenzwert (mg/l)"
]

df_adriatische_forelle = df_adriatische_forelle.drop(columns=spalten_entfernen, errors="ignore")

display(df_adriatische_forelle)

df_adriatische_forelle

In [ ]:
seen_der_aesche = df_adriatische_forelle["lake"].unique()
print("Seen mit Adriatischer Forelle:", seen_der_aesche)

In [ ]:
# Einheitlich schreiben
df_merged["depth"] = df_merged["depth"].str.strip().str.lower()

# Alle Fische
fische = df_merged["Fisch"].dropna().unique()

# Ergebnisliste
fisch_stats = []

for fisch in fische:
    # Alle Seen mit diesem Fisch
    alle_seen = df_merged[df_merged["Fisch"] == fisch]["lake"].unique()
    
    # Seen, in denen er irgendwo ausstirbt
    seen_mit_aussterben = df_merged[
        (df_merged["Fisch"] == fisch) &
        (df_merged["temperature_avg"] > df_merged["Kritische Temperatur °C"]) &
        (df_merged["depth"] == "surface")
    ]["lake"].unique()
    
    fisch_stats.append({
        "Fisch": fisch,
        "Anzahl_Seen": len(alle_seen),
        "Anzahl_Seen_mit_Aussterben": len(seen_mit_aussterben)
    })

# Als DataFrame
df_fisch_übersicht = pd.DataFrame(fisch_stats).sort_values(by="Anzahl_Seen_mit_Aussterben", ascending=False)

display(df_fisch_übersicht)

In [ ]:
#alle infos zu Trügen

fisch = "Adriatische Forelle"

df_forelle = df_merged[
    (df_merged["Fisch"] == fisch) &
    (df_merged["depth"].str.strip().str.lower() == "surface")
].copy()

seen_forelle = df_forelle["lake"].unique()
print("Seen:", seen_forelle)


for see in seen_forelle:
    df_plot = df_forelle[df_forelle["lake"] == see]
    df_plot_grouped = df_plot.groupby(["year", "scenario"]).agg({
        "temperature_avg": "mean",
        "Kritische Temperatur °C": "first"
    }).reset_index()

    plt.figure(figsize=(12, 6))

    for scenario in df_plot_grouped["scenario"].unique():
        gruppe = df_plot_grouped[df_plot_grouped["scenario"] == scenario].sort_values("year")
        plt.plot(gruppe["year"], gruppe["temperature_avg"],
                 label=labels.get(scenario, scenario),
                 color=farben.get(scenario), linewidth=2)

        krit_temp = gruppe["Kritische Temperatur °C"].iloc[0]
        dead = gruppe[gruppe["temperature_avg"] > krit_temp]
        if not dead.empty:
            year_dead = dead["year"].iloc[0]
            temp_dead = dead["temperature_avg"].iloc[0]
            plt.scatter(year_dead, temp_dead, color=farben[scenario], s=60)
            plt.text(year_dead + 1, temp_dead + 0.2,
                     f"ab {year_dead}", color=farben[scenario], fontsize=9)

    plt.axhline(krit_temp, color="red", linestyle=":", label=f"Krit. Temp. ({krit_temp:.1f} °C)")
    plt.title(f"{fisch} in {see} – Temperaturverlauf (surface)")
    plt.xlabel("Jahr")
    plt.ylabel("Ø Temperatur [°C]")
    plt.legend()
    plt.tight_layout()
    plt.show()

aussterben_liste = []

for see in seen_forelle:
    for scenario in ["RCP26", "RCP45", "RCP85"]:
        gruppe = df_forelle[
            (df_forelle["lake"] == see) &
            (df_forelle["scenario"] == scenario)
        ].groupby("year").agg({
            "temperature_avg": "mean",
            "Kritische Temperatur °C": "first"
        }).reset_index()

        if gruppe.empty or gruppe["Kritische Temperatur °C"].isna().all():
            continue

        krit_temp = gruppe["Kritische Temperatur °C"].iloc[0]
        dead = gruppe[gruppe["temperature_avg"] > krit_temp]
        year = dead["year"].iloc[0] if not dead.empty else "überlebt"

        aussterben_liste.append({
            "Fisch": fisch,
            "See": see,
            "Szenario": scenario,
            "Aussterbejahr": year
        })

df_forelle_aussterben = pd.DataFrame(aussterben_liste)
display(df_forelle_aussterben)

In [ ]:
# Liste der relevanten Seen
relevante_seen = ["Maggiore", "Lower-Lugano", "Upper-Lugano"]

# Filter auf Oberfläche und relevante Seen
df_filtered = df_merged[
    (df_merged["depth"].str.strip().str.lower() == "surface") &
    (df_merged["lake"].isin(relevante_seen))
].copy()

# Liste zum Speichern der Aussterbedaten
aussterben_liste = []

# Schleife über Fischarten und Seen
for (fisch, see) in df_filtered.groupby(["Fisch", "lake"]).groups.keys():
    for scenario in ["RCP26", "RCP45", "RCP85"]:
        gruppe = df_filtered[
            (df_filtered["Fisch"] == fisch) &
            (df_filtered["lake"] == see) &
            (df_filtered["scenario"] == scenario)
        ].groupby("year").agg({
            "temperature_avg": "mean",
            "Kritische Temperatur °C": "first"
        }).reset_index()

        if gruppe.empty or gruppe["Kritische Temperatur °C"].isna().all():
            continue

        krit_temp = gruppe["Kritische Temperatur °C"].iloc[0]
        dead = gruppe[gruppe["temperature_avg"] > krit_temp]
        
        # Nur aufnehmen, wenn ein Aussterbejahr existiert
        if not dead.empty:
            year = dead["year"].iloc[0]
            aussterben_liste.append({
                "See": see,
                "Fisch": fisch,
                "Szenario": scenario,
                "Aussterbejahr": year
            })

# DataFrame mit nur aussterbenden Fischen
df_aussterben_only = pd.DataFrame(aussterben_liste)
display(df_aussterben_only)

In [ ]:
# Liste der relevanten Seen
relevante_seen = ["Maggiore", "Lower-Lugano", "Upper-Lugano"]

# Filter auf Oberfläche und relevante Seen
df_filtered = df_merged[
    (df_merged["depth"].str.strip().str.lower() == "surface") &
    (df_merged["lake"].isin(relevante_seen))
].copy()

# Liste zum Speichern der Aussterbedaten
aussterben_liste = []

# Schleife über Fischarten und Seen
for (fisch, see) in df_filtered.groupby(["Fisch", "lake"]).groups.keys():
    for scenario in ["RCP26", "RCP45", "RCP85"]:
        gruppe = df_filtered[
            (df_filtered["Fisch"] == fisch) &
            (df_filtered["lake"] == see) &
            (df_filtered["scenario"] == scenario)
        ].groupby("year").agg({
            "temperature_avg": "mean",
            "Kritische Temperatur °C": "first"
        }).reset_index()

        if gruppe.empty or gruppe["Kritische Temperatur °C"].isna().all():
            continue

        krit_temp = gruppe["Kritische Temperatur °C"].iloc[0]
        dead = gruppe[gruppe["temperature_avg"] > krit_temp]

        if not dead.empty:
            year = dead["year"].iloc[0]
            aussterben_liste.append({
                "Fisch": fisch,
                "See": see,
                "Szenario": scenario,
                "Aussterbejahr": year
            })

# DataFrame mit nur aussterbenden Fischen
df_aussterben_only = pd.DataFrame(aussterben_liste)

# Eine Tabelle je Fisch (als Dictionary)
fisch_tabellen = {
    fisch: df_aussterben_only[df_aussterben_only["Fisch"] == fisch].reset_index(drop=True)
    for fisch in df_aussterben_only["Fisch"].unique()
}

# Ausgabe je Fisch
for name, df in fisch_tabellen.items():
    print(f"\n===== {name} =====")
    display(df)

In [ ]:
# 1. Adriatische Forelle entfernen
df_bereinigt = df_aussterben_only[df_aussterben_only["Fisch"] != "Adriatische Forelle"].copy()

# 2. Nur die gewünschten Spalten behalten
df_bereinigt = df_bereinigt[["See", "Fisch", "Szenario", "Aussterbejahr"]]

# 3. Sortieren
df_bereinigt = df_bereinigt.sort_values(by=["See", "Fisch", "Aussterbejahr"])

# 4. Gruppiert anzeigen (z. B. im Notebook)
for see in df_bereinigt["See"].unique():
    print(f" {see}")
    display(df_bereinigt[df_bereinigt["See"] == see][["Fisch", "Szenario", "Aussterbejahr"]])
    print("-" * 50 + "\n")

In [ ]:
# Nur Oberflächenwerte verwenden
df_surface = df_merged[df_merged["depth"].str.strip().str.lower() == "surface"].copy()

# Gruppiere nach See, rechne Durchschnittstemperatur
df_avg_temp = df_surface.groupby("lake")["temperature_avg"].mean().reset_index()

# Sortiere nach Temperatur (absteigend)
df_avg_temp_sorted = df_avg_temp.sort_values(by="temperature_avg", ascending=False)

# Zeige die Top 1 oder mehrere (z. B. Top 5)
waermster_see = df_avg_temp_sorted.iloc[0]
print("Wärmster See der Schweiz:")
print(waermster_see)

# ganze sortierte Liste anzeigen
display(df_avg_temp_sorted)

In [ ]:
# Filter für RCP45, surface und relevante Jahre (angepasst)
df_filtered = df_merged[
    (df_merged["depth"].str.strip().str.lower() == "surface") &
    (df_merged["scenario"] == "RCP45") &
    (df_merged["year"].isin([1981, 2025, 2026, 2099]))
]

# Pivotieren: See = Zeile, Jahr = Spalte
df_pivot = df_filtered.pivot_table(index="lake", columns="year", values="temperature_avg")

# Temperaturveränderungen berechnen
df_pivot["Δ 1981–2025"] = df_pivot[2025] - df_pivot[1981]
df_pivot["Δ 2026–2099"] = df_pivot[2099] - df_pivot[2026]

# Ausgabe
df_temp_change = df_pivot[["Δ 1981–2025", "Δ 2026–2099"]].reset_index()
display(df_temp_change)

In [ ]:
# Hier deine Parameter setzen:
see_name = "Upper-Lugano"
fisch_name = "Adriatische Forelle"

# Kritische Temperatur aus df_merged herauslesen
krit_temp = df_merged.loc[
    (df_merged["Fisch"] == fisch_name) & 
    (df_merged["lake"] == see_name)
]["Kritische Temperatur °C"].dropna().unique()

if len(krit_temp) == 0:
    print(f"Keine kritische Temperatur für {fisch_name} in {see_name} gefunden.")
    krit_temp = [np.nan]
krit_temp = krit_temp[0]

# Daten vorbereiten
df_see = df_merged[
    (df_merged["lake"] == see_name) &
    (df_merged["depth"].str.lower() == "surface")  # falls 'depth' existiert
].copy()

plt.figure(figsize=(12, 6))

for scenario in ["RCP26", "RCP45", "RCP85"]:
    df_szen = df_see[df_see["scenario"] == scenario].copy()
    df_mean = df_szen.groupby("year")["temperature_avg"].mean().reset_index()

    # Temperatur glätten
    df_mean["temp_smooth"] = df_mean["temperature_avg"].rolling(window=10, center=True, min_periods=1).mean()

    # Simuliertes geglättetes Rauschen
    np.random.seed(42 + hash(scenario + see_name) % 1000)
    noise = np.random.normal(0, 0.3, size=len(df_mean))
    noise_smooth = pd.Series(noise).rolling(window=21, center=True, min_periods=1).mean()

    # Unsicherheitsband berechnen
    df_mean["lower"] = df_mean["temp_smooth"] - abs(noise_smooth)
    df_mean["upper"] = df_mean["temp_smooth"] + abs(noise_smooth)

    # Plot Linie und Band
    plt.plot(df_mean["year"], df_mean["temp_smooth"], label=labels[scenario], color=farben[scenario], linewidth=2)
    plt.fill_between(df_mean["year"], df_mean["lower"], df_mean["upper"],
                     color=farben[scenario], alpha=0.2)

    # Unsicheres Aussterbeintervall berechnen
    cross_start = df_mean[df_mean["lower"] > krit_temp]
    #cross_start = df_mean[df_mean["upper"] > krit_temp]
    cross_end = df_mean[df_mean["upper"] > krit_temp]

    if not cross_start.empty and not cross_end.empty:
        year_start = cross_start["year"].iloc[0]
        year_end = cross_end["year"].iloc[0]

        # Intervall visuell darstellen
        plt.axvspan(year_start, year_end, color=farben[scenario], alpha=0.15)
        plt.text((year_start + year_end) / 2, krit_temp + 0.3,
                 f"Risiko {int(year_end)}–{int(year_start)}", color=farben[scenario],
                 fontsize=9, ha="center", va="bottom")

# Horizontale Linie: Kritische Temperatur
plt.axhline(krit_temp, color="red", linestyle=":", linewidth=1.5, label=f"Krit. Temp. ({krit_temp:.1f} °C)")

# Layout
plt.title(f"{fisch_name} in {see_name} – Temperaturverlauf & Aussterberisiko-Intervall")
plt.xlabel("Jahr")
plt.ylabel("Ø Temperatur [°C]")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(title="Szenario")
plt.tight_layout()
plt.show()

In [ ]:
# Parameter
see_name = "Upper-Lugano"
fisch_name = "Adriatische Forelle"

# Kritische Temperatur extrahieren
krit_temp = df_merged.loc[
    (df_merged["Fisch"] == fisch_name) &
    (df_merged["lake"] == see_name)
]["Kritische Temperatur °C"].dropna().unique()

krit_temp = float(krit_temp[0]) if len(krit_temp) > 0 else np.nan

# Filter für den gewünschten See und Tiefe
df_see = df_merged[
    (df_merged["lake"] == see_name) &
    (df_merged["depth"].str.lower() == "surface")
].copy()

plt.figure(figsize=(12, 6))

for scenario in ["RCP26", "RCP45", "RCP85"]:
    df_szen = df_see[df_see["scenario"] == scenario].copy()
    df_mean = df_szen.groupby("year")["temperature_avg"].mean().reset_index()
    
    # Temperatur glätten
    df_mean["temp_smooth"] = df_mean["temperature_avg"].rolling(window=10, center=True, min_periods=1).mean()
    
    # Rauschen simulieren und glätten
    np.random.seed(42 + hash(scenario + see_name) % 1000)
    noise = np.random.normal(0, 0.3, size=len(df_mean))
    noise_smooth = pd.Series(noise).rolling(window=21, center=True, min_periods=1).mean().values

    df_mean["lower"] = df_mean["temp_smooth"] - np.abs(noise_smooth)
    df_mean["upper"] = df_mean["temp_smooth"] + np.abs(noise_smooth)

    # Plot Temperaturkurve und Unsicherheitsbereich
    plt.plot(df_mean["year"], df_mean["temp_smooth"], label=labels[scenario], color=farben[scenario])
    plt.fill_between(df_mean["year"], df_mean["lower"], df_mean["upper"],
                     color=farben[scenario], alpha=0.2)

    # Nur Kontaktintervall markieren
    mask_contact = (df_mean["lower"] <= krit_temp) & (df_mean["upper"] >= krit_temp)
    if mask_contact.any():
        jahr_start = df_mean["year"][mask_contact].iloc[0]
        jahr_ende = df_mean["year"][mask_contact].iloc[-1]
        plt.axvspan(jahr_start, jahr_ende, color=farben[scenario], alpha=0.15)
        plt.text((jahr_start + jahr_ende)/2, krit_temp + 0.3,
                 f"Kontakt {int(jahr_start)}–{int(jahr_ende)}",
                 color=farben[scenario], fontsize=9, ha="center")

# Horizontale Linie: kritische Temperatur
plt.axhline(krit_temp, color="red", linestyle="--", linewidth=1.5,
            label=f"Krit. Temp. ({krit_temp:.1f} °C)")

# Layout
plt.suptitle(f"{fisch_name} in {see_name}", size= 14)
plt.title("Temperaturverlauf & Kontaktintervall zur kritischen Schwelle")
plt.xlabel("Jahr")
plt.ylabel("Ø Temperatur [°C]")
plt.legend(title="Szenario")
plt.tight_layout()
plt.show()

Meine Datenstory richtet sich an ein breites, umweltinteressiertes Publikum, das sich für die Auswirkungen des Klimawandels auf die Biodiversität in Schweizer Seen interessiert. Besonders geeignet ist sie für gebildete Laien, Umweltorganisationen sowie Entscheidungsträger:innen, die evidenzbasierte Darstellungen benötigen, um Handlungsspielräume zu erkennen.

In [ ]:
window = 10  # Rolling-Window in Jahren
fisch = "Adriatische Forelle"

for see in seen_forelle:
    df_plot = df_forelle[df_forelle["lake"] == see]
    df_plot_grouped = df_plot.groupby(["year", "scenario"]).agg({
        "temperature_avg": "mean",
        "Kritische Temperatur °C": "first"
    }).reset_index()

    plt.figure(figsize=(14, 6))

    for scenario in df_plot_grouped["scenario"].unique():
        gruppe = df_plot_grouped[df_plot_grouped["scenario"] == scenario].sort_values("year")

        # Rolling-Werte
        rolling_avg = gruppe["temperature_avg"].rolling(window=window, min_periods=1).mean()
        rolling_std = gruppe["temperature_avg"].rolling(window=window, min_periods=1).std()

        years = gruppe["year"]
        farbe = farben.get(scenario, "gray")

        # Hauptlinie
        plt.plot(years, rolling_avg,
                 label=labels.get(scenario, scenario),
                 color=farbe, linewidth=2.5)

        # Unsicherheitsband (grau mit Farbkante)
        plt.fill_between(years,
                         rolling_avg - rolling_std,
                         rolling_avg + rolling_std,
                         color="gray", alpha=0.15)

        # Kritischer Punkt
        krit_temp = gruppe["Kritische Temperatur °C"].iloc[0]
        dead = gruppe[rolling_avg > krit_temp]
        if not dead.empty:
            year_dead = dead["year"].iloc[0]
            temp_dead = rolling_avg[dead.index[0]]
            plt.scatter(year_dead, temp_dead, color=farbe, s=60)
            plt.text(year_dead + 1, temp_dead + 0.2,
                     f"ab {year_dead}", color=farbe, fontsize=9)

    # Kritische Temperaturlinie
    plt.axhline(krit_temp, color="red", linestyle=":", linewidth=1.5,
                label=f"Krit. Temp. ({krit_temp:.1f} °C)")

    plt.suptitle(f"{fisch} in {see}", fontsize=14)
    plt.title("Temperaturverlauf der Klimaszenarien, geglättet über 10 Jahre", fontsize=11)
    plt.xlabel("Jahr")
    plt.ylabel("Ø Temperatur [°C]")
    plt.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

In [ ]:
window = 10
fisch = "Adriatische Forelle"

for see in seen_forelle:
    df_plot = df_forelle[df_forelle["lake"] == see]
    df_plot_grouped = df_plot.groupby(["year", "scenario"]).agg({
        "temperature_avg": "mean",
        "Kritische Temperatur °C": "first"
    }).reset_index()

    plt.figure(figsize=(14, 6))

    for scenario in df_plot_grouped["scenario"].unique():
        gruppe = df_plot_grouped[df_plot_grouped["scenario"] == scenario].sort_values("year")
        years = gruppe["year"].to_numpy()
        krit_temp = gruppe["Kritische Temperatur °C"].iloc[0]
        farbe = farben.get(scenario, "gray")

        rolling_avg = gruppe["temperature_avg"].rolling(window=window, min_periods=1).mean().to_numpy()
        rolling_std = gruppe["temperature_avg"].rolling(window=window, min_periods=1).std().to_numpy()
        upper = rolling_avg + rolling_std
        lower = rolling_avg - rolling_std

        # 1. Grauer Grundbereich ohne Lücken
        plt.fill_between(years, lower, upper, color="gray", alpha=0.15)

        # 2. Farbiger Bereich oberhalb der kritischen Temperatur
        mask = upper > krit_temp
        plt.fill_between(years,
                         np.where(mask, np.maximum(lower, krit_temp), np.nan),
                         np.where(mask, upper, np.nan),
                         color=farbe, alpha=0.3)

        # Hauptlinie
        plt.plot(years, rolling_avg, label=labels.get(scenario, scenario), color=farbe, linewidth=2.5)

        # Kritischer Punkt
        exceed_index = np.argmax(rolling_avg > krit_temp)
        if rolling_avg[exceed_index] > krit_temp:
            year_dead = years[exceed_index]
            temp_dead = rolling_avg[exceed_index]
            plt.scatter(year_dead, temp_dead, color=farbe, s=60)
            plt.text(year_dead + 1, temp_dead + 0.2, f"ab {year_dead}", color=farbe, fontsize=9)

    # Kritische Linie
    plt.axhline(krit_temp, color="red", linestyle=":", linewidth=1.5,
                label=f"Krit. Temp. ({krit_temp:.1f} °C)")

    plt.suptitle(f"{fisch} in {see}", fontsize=14)
    plt.title("Temperaturverlauf der Klimaszenarien, geglättet über 10 Jahre", fontsize=11)
    plt.xlabel("Jahr")
    plt.ylabel("Ø Temperatur [°C]")
    plt.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

In [ ]:
# Filter nur Oberfläche
df_surface = df_merged[df_merged['depth'] == 'surface']

# Ziel-Szenarien
szenarien = ['RCP26', 'RCP45', 'RCP85']
farben = {'RCP26': 'green', 'RCP45': 'orange', 'RCP85': 'red'}

# Liste zur Speicherung
steigerung_liste = []

# Für jedes Szenario und jeden See: Ø Temp für beide Zeiträume berechnen
for szenario in szenarien:
    df_szen = df_surface[df_surface['scenario'] == szenario]
    for see in df_szen['lake'].unique():
        df_see = df_szen[df_szen['lake'] == see]
        temp_1980_2024 = df_see[(df_see['year'] >= 1980) & (df_see['year'] <= 2024)]['temperature_avg'].mean()
        temp_2025_2099 = df_see[(df_see['year'] >= 2025) & (df_see['year'] <= 2099)]['temperature_avg'].mean()
        if pd.notnull(temp_1980_2024) and pd.notnull(temp_2025_2099):
            differenz = temp_2025_2099 - temp_1980_2024
            steigerung_liste.append({'see': see, 'szenario': szenario, 'temp_diff': differenz})

# In DataFrame umwandeln
df_steigerung = pd.DataFrame(steigerung_liste)

# --- Plot als Balkendiagramm ---
plt.figure(figsize=(14, 6))
for szenario in szenarien:
    df_plot = df_steigerung[df_steigerung['szenario'] == szenario]
    plt.bar(
        [f"{row['see']} ({szenario})" for _, row in df_plot.iterrows()],
        df_plot['temp_diff'],
        label=szenario,
        color=farben[szenario],
        alpha=0.7
    )

plt.xticks(rotation=90)
plt.ylabel("Temperaturanstieg [°C]")
plt.title("Durchschnittlicher Temperaturanstieg pro See und Szenario (2099 vs. 2024)")
plt.legend(title="Szenario")
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Mapping der Szenarien auf sprechende Namen
szenarien_namen = {
    "RCP26": "Best-Case",
    "RCP45": "Middle-Case",
    "RCP85": "Worst-Case"
}

plt.figure(figsize=(14, 6))

seen = df_steigerung['see'].unique()
x = np.arange(len(seen))  # Positionen für die Gruppen
breite = 0.25  # Balkenbreite

for i, szenario in enumerate(szenarien):
    df_plot = df_steigerung[df_steigerung['szenario'] == szenario].set_index('see').reindex(seen)
    plt.bar(
        x + i * breite,
        df_plot['temp_diff'],
        width=breite,
        label=szenarien_namen[szenario],  # <-- neue Namen hier
        color=farben[szenario],
        alpha=0.7
    )

plt.xticks(x + breite, seen, rotation=90)
plt.ylabel("Temperaturanstieg [°C]")
plt.title("Durchschnittlicher Temperaturanstieg pro See nach Szenario (2025–2099 vs. 1981–2024)")
plt.legend(title="Szenario")
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()

plt.savefig("img/temp-vorhersage.png", dpi=300)
plt.show()